In [3]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import certifi,csv,time
import warnings
import random
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd

In [138]:
movie_info = ["movie_name","actors_List","genre_list"]
data_list = []
data_list_iterator = 0


In [139]:
#Defining the list of base urls to scrap
url_list = ["https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc",
           "https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc&page=1",
           "https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc&page=2",
           "https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc&page=3",
           "https://www.metacritic.com/browse/movies/score/metascore/all/filtered?sort=desc&page=4"]

In [248]:
specific_movies_links = []
messy_links = []
for url in url_list:
    bs=BeautifulSoup(urlopen(Request(url, headers={'User-Agent': 'Mozilla/5.0'}),cafile=certifi.where()).read())
    temporary_messy = list(bs.find_all("tr"))
    print(len(temporary_messy))
    messy_links.append(temporary_messy)
#specific_movies_links = ["https://www.metacritic.com"+temporary_messy[i].a.attrs["href"]+"/details" for i in range(0,len(temporary_messy),2)]


199
199
199
199
199


In [249]:
for link_list in messy_links:
    for i  in range(0,len(link_list),2):
        specific_movies_links.append("https://www.metacritic.com"+link_list[i].a.attrs["href"]+"/details")

In [269]:
new_messy = []
for link_list in messy_links:
    for individual_url in link_list:
        new_messy.append(individual_url)
    new_messy.append("I am a placeholder")
messy_links = new_messy

In [58]:
def scrap_movie_details(movie_url):
    global data_list_iterator, data_list
    bs_movie = BeautifulSoup(urlopen(Request(movie_url, headers={'User-Agent': 'Mozilla/5.0'}),cafile=certifi.where()).read())
    genres = bs_movie.find_all("tr",{"class":"genres"})[0].find_all("span")
    genres_list = [value.get_text().strip() for value in genres]
    movie_name = messy_links[data_list_iterator].h3.get_text().lower()
    temp_actors = []
    actors_data= {}
    actors_raw_data = bs_movie.find_all("table",{"class":"credits"})[2:4]
    for data in actors_raw_data:
        td_List = data.find_all("td")
        for name in td_List:
            temp_actors.append(name.get_text().strip())
    for i in range(0,len(temp_actors),2):
        actors_data[temp_actors[i]]=temp_actors[i+1]
    data_list.append({"movie_name":movie_name,"genre_list":genres_list,"actors_List":actors_data})
    data_list_iterator += 2      

In [274]:
start = 0
end = len(specific_movies_links)

In [292]:
for i in range(start,end):
    scrap_movie_details(specific_movies_links[i])
    start += 1
    if start>=end:
        print("All data collected")
        break

All data collected


In [307]:
with open('movie_data.csv', 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames = movie_info)
    writer.writeheader()
    writer.writerows(data_list)

In [4]:
import ast
from scipy import spatial
df = pd.read_csv("movie_data.csv",encoding='latin-1')
item_to_check=input("What do you want to check on Meticritics? Please choose 'movie','people' or 'comparison' ").lower()

def check_person(their_name):
    movies_acted = df.loc[df["actors_List"].str.contains(their_name,case=False)]['movie_name']
    people_str = f"{their_name} has acted in "
    people_genre_str = f"His most often played genres are "
    grab_genres = df.loc[df["actors_List"].str.contains(their_name,case=False)]['genre_list']
    genres_acted = []
    for item in grab_genres:
        genres_acted.extend(ast.literal_eval(item))
    people_genre_str = f"His most often played genres are "
    genres_acted_set = list(set(genres_acted))
    for genre in genres_acted_set:
        people_genre_str += genre+f" :{genres_acted.count(genre)} "
    for i in movies_acted:
        people_str += i+", "
    people_str += "...etc"
    print(people_str)
    print(people_genre_str)
    return genres_acted
    
if item_to_check=="movie":
    specific_item = input(f'What {item_to_check} do you want to check  ').title()
    genre_string =f"The genre of the movie is "
    actors = df.loc[df['movie_name'] == specific_item]["actors_List"]
    genres = df.loc[df['movie_name'] == specific_item]["genre_list"]
    for names in actors:
        actors_string = names
    actors_string = actors_string.replace("\'", "\"")
    actors_list = actors_string.split(",")
    string_result = f"\n\nThe cast of the movie {specific_item}  includes "
    genre_list = ast.literal_eval([value for value in genres][0])
    genre_length = len(genre_list)
    for i in range(genre_length):
        if i < genre_length-1:
            genre_string += genre_list[i]+", "
        else:
            genre_string += genre_list[i]+"."
    for value in actors_list:
        string_result+=value[2:-2].split(":")[0]+ " as "+value[2:-2].split(":")[1]+','
    print(string_result)
    print("\n\n"+genre_string)
elif item_to_check == "people":
    name = input("Who do you want to check?")
    check_person(name.title())
elif item_to_check == "comparison":
    print("Who do you want to compare? ")
    first_person = input("Enter the name of the first person: ")
    second_person = input("Enter the name of the second person: ")
    first_person_genres = check_person(first_person)
    second_person_genres = check_person(second_person)
    raw_genres_list = []
    raw_genres_list.extend(second_person_genres)
    raw_genres_list.extend(first_person_genres)
    genres_total_list = []
    first_person_array =[]
    second_person_array = []
    genres_set = set(raw_genres_list)
    genres_set = sorted(list(genres_set))
    for value in genres_set:
        first_person_array.append(first_person_genres.count(value))
        second_person_array.append(second_person_genres.count(value))
    cosine_similarity = 1 - spatial.distance.cosine(first_person_array, second_person_array)
    print(f"Based on that, they have a cosine similarity score of {cosine_similarity}")
else:
    print("Please you have to enter either 'people','movie' or 'comparison to proceed'")

What do you want to check on Meticritics? Please choose 'movie','people' or 'comparison' comparison
Who do you want to compare? 
Enter the name of the first person: Joseph Cotten
Enter the name of the second person: Frank Cady
Joseph Cotten has acted in Citizen Kane, The Third Man, Shadow of a Doubt, The Magnificent Ambersons, F for Fake, ...etc
His most often played genres are Thriller :2 Documentary :1 Drama :2 Mystery :2 Romance :1 Film-Noir :1 
Frank Cady has acted in Rear Window, ...etc
His most often played genres are Thriller :1 Mystery :1 
Based on that, they have a cosine similarity score of 0.7302967433402215
